In [1]:
versioninfo()

Julia Version 1.9.3
Commit bed2cd540a1 (2023-08-24 14:43 UTC)
Build Info:
  Official https://julialang.org/ release
Platform Info:
  OS: macOS (x86_64-apple-darwin22.4.0)
  CPU: 8 × Intel(R) Core(TM) i5-8279U CPU @ 2.40GHz
  WORD_SIZE: 64
  LIBM: libopenlibm
  LLVM: libLLVM-14.0.6 (ORCJIT, skylake)
  Threads: 2 on 8 virtual cores


In [2]:
using Pkg
Pkg.activate(pwd())
Pkg.instantiate()
Pkg.status()

  Activating project at `~/Dropbox/class/M1399.000200/2023/M1300_000200-2023fall/lectures/11-iterative`


Status `~/Dropbox/class/M1399.000200/2023/M1300_000200-2023fall/lectures/11-iterative/Project.toml`
  [6e4b80f9] BenchmarkTools v1.3.2
  [42fd0dbc] IterativeSolvers v0.9.2
  [b51810bb] MatrixDepot v1.0.10
  [af69fa37] Preconditioners v0.6.1
  [b8865327] UnicodePlots v3.6.0


## Introduction

So far we have considered direct methods for solving linear equations.    

* **Direct methods** (flops fixed _a priori_) vs **iterative methods**:
    - Direct method (GE/LU, Cholesky, QR, SVD): accurate. good for dense, small to moderate sized $\mathbf{A}$  
    - Iterative methods (Jacobi, Gauss-Seidal, SOR, conjugate-gradient, GMRES): accuracy depends on number of iterations. good for large, sparse, structured linear system, parallel computing, warm start (reasonable accuracy after, say, 100 iterations)

## {background-color="white"}

### Motivation: PageRank 

<img src="https://upload.wikimedia.org/wikipedia/commons/thumb/f/fb/PageRanks-Example.svg/400px-PageRanks-Example.svg.png" width="400" align="center"/>

Source: [Wikepedia](https://en.wikipedia.org/wiki/PageRank)

---

* $\mathbf{A}  \in \{0,1\}^{n \times n}$ the connectivity matrix of webpages with entries
$$
\begin{eqnarray*}
	a_{ij} = \begin{cases}
	1 &  \text{if page $i$ links to page $j$} \\
	0 & \text{otherwise}
	\end{cases}. 
\end{eqnarray*}
$$
$n \approx 10^9$ in May 2017.

* $r_i = \sum_j a_{ij}$ is the *out-degree* of page $i$. 

---

* [Larry Page](https://en.wikipedia.org/wiki/PageRank) imagined a random surfer wandering on internet according to following rules:
    - From a page $i$ with $r_i>0$
        * with probability $p$, (s)he randomly chooses a link $j$ from page $i$ (uniformly) and follows that link to the next page  
        * with probability $1-p$, (s)he randomly chooses one page from the set of all $n$ pages (uniformly) and proceeds to that page 
    - From a page $i$ with $r_i=0$ (a dangling page), (s)he randomly chooses one page from the set of all $n$ pages (uniformly) and proceeds to that page
    
---

* The process defines an $n$-state Markov chain, where each state corresponds to each page. 
$$
    p_{ij} = (1-p)\frac{1}{n} + p\frac{a_{ij}}{r_i}
$$
with interpretation $a_{ij}/r_i = 1/n$ if $r_i = 0$.

* Stationary distribution of this Markov chain gives the score (long term probability of visit) of each page.

* Stationary distribution can be obtained as the top *left* eigenvector of the transition matrix $\mathbf{P}=(p_{ij})$ corresponding to eigenvalue 1. 
$$
    \mathbf{x}^T\mathbf{P} = \mathbf{x}^T.
$$

---

* Equivalently it can be cast as a linear equation.
$$
    (\mathbf{I} - \mathbf{P}^T) \mathbf{x} = \mathbf{0}.
$$

* You've got to solve a linear equation with $10^9$ variables!

* GE/LU will take $2 \times (10^9)^3/3/10^{12} \approx 6.66 \times 10^{14}$ seconds $\approx 20$ million years on a tera-flops supercomputer!

* Iterative methods come to the rescue.

## Splitting and fixed point iteration

* The key idea of iterative method for solving $\mathbf{A}\mathbf{x}=\mathbf{b}$ is to **split** the matrix $\mathbf{A}$ so that
$$
    \mathbf{A} = \mathbf{M} - \mathbf{K}
$$
where $\mathbf{M}$ is invertible and easy to invert.

* Then $\mathbf{A}\mathbf{x} = \mathbf{M}\mathbf{x} - \mathbf{K}\mathbf{x} = \mathbf{b}$ or
$$
    \mathbf{x} = \mathbf{M}^{-1}\mathbf{K}\mathbf{x} - \mathbf{M}^{-1}\mathbf{b}
    .
$$
Thus a solution to $\mathbf{A}\mathbf{x}=\mathbf{b}$ is a fixed point of iteration
$$
    \mathbf{x}^{(t+1)} = \mathbf{M}^{-1}\mathbf{K}\mathbf{x}^{(t)} - \mathbf{M}^{-1}\mathbf{b}
    = \mathbf{R}\mathbf{x}^{(t)} - \mathbf{c}
    .
$$

---

* Under a suitable choice of $\mathbf{R}$, i.e., splitting of $\mathbf{A}$, the sequence $\mathbf{x}^{(t)}$ generated by the above iteration converges to a solution $\mathbf{A}\mathbf{x}=\mathbf{b}$:

Theorem

> Let $\rho(\mathbf{R})=\max_{i=1,\dotsc,n}|\lambda_i(\mathbf{R})|$, where $\lambda_i(\mathbf{R})$ is the $i$th (complex) eigenvalue of $\mathbf{R}$. The iteration $\mathbf{x}^{(t+1)}=\mathbf{R}\mathbf{x}^{(t)} - \mathbf{c}$ converges to a solution to $\mathbf{A}\mathbf{x}=\mathbf{b}$ if and only if $\rho(\mathbf{R}) < 1$.

* Proof: HW.

## Jacobi's method

$$
x_i^{(t+1)} = \frac{b_i - \sum_{j=1}^{i-1} a_{ij} x_j^{(t)} - \sum_{j=i+1}^n a_{ij} x_j^{(t)}}{a_{ii}}.
$$


* Split $\mathbf{A} = \mathbf{L} + \mathbf{D} + \mathbf{U}$ = (strictly lower triangular) + (diagonal) + (strictly upper triangular).


* Take $\mathbf{M}=\mathbf{D}$ (easy to invert!) and $\mathbf{K}=-(\mathbf{L} + \mathbf{U})$:
$$
    \mathbf{D} \mathbf{x}^{(t+1)} = - (\mathbf{L} + \mathbf{U}) \mathbf{x}^{(t)} + \mathbf{b},
$$
i.e., 
$$
	\mathbf{x}^{(t+1)} = - \mathbf{D}^{-1} (\mathbf{L} + \mathbf{U}) \mathbf{x}^{(t)} + \mathbf{D}^{-1} \mathbf{b} = - \mathbf{D}^{-1} \mathbf{A} \mathbf{x}^{(t)} + \mathbf{x}^{(t)} + \mathbf{D}^{-1} \mathbf{b}.
$$

---

* Convergence is guaranteed if $\mathbf{A}$ is striclty row diagonally dominant: $|a_{ii}| > \sum_{j\neq i}|a_{ij}|$.

* One round costs $2n^2$ flops with an **unstructured** $\mathbf{A}$. Gain over GE/LU if converges in $o(n)$ iterations. 

* Saving is huge for **sparse** or **structured** $\mathbf{A}$. By structured, we mean the matrix-vector multiplication $\mathbf{A} \mathbf{v}$ is fast ($O(n)$ or $O(n \log n)$).
    - Often the multiplication is implicit and $\mathbf{A}$ is not even stored, e.g., finite difference: $(\mathbf{A}\mathbf{v})_i = v_i - v_{i+1}$.

## Gauss-Seidel method

$$
x_i^{(t+1)} = \frac{b_i - \sum_{j=1}^{i-1} a_{ij} x_j^{(t+1)} - \sum_{j=i+1}^n a_{ij} x_j^{(t)}}{a_{ii}}.
$$

* Split $\mathbf{A} = \mathbf{L} + \mathbf{D} + \mathbf{U}$ = (strictly lower triangular) + (diagonal) + (strictly upper triangular) as Jacobi.

* Take $\mathbf{M}=\mathbf{D}+\mathbf{L}$ (easy to invert, why?) and $\mathbf{K}=-\mathbf{U}$:
$$
(\mathbf{D} + \mathbf{L}) \mathbf{x}^{(t+1)} = - \mathbf{U} \mathbf{x}^{(t)} + \mathbf{b}
$$
i.e., 
$$
\mathbf{x}^{(t+1)} = - (\mathbf{D} + \mathbf{L})^{-1} \mathbf{U} \mathbf{x}^{(t)} + (\mathbf{D} + \mathbf{L})^{-1} \mathbf{b}.
$$

---

* Equivalent to
$$
\mathbf{D}\mathbf{x}^{(t+1)} = - \mathbf{L} \mathbf{x}^{(t+1)} - \mathbf{U} \mathbf{x}^{(t)} + \mathbf{b}
$$
or
$$
\mathbf{x}^{(t+1)} = \mathbf{D}^{-1}(- \mathbf{L} \mathbf{x}^{(t+1)} - \mathbf{U} \mathbf{x}^{(t)} + \mathbf{b})
$$
leading to the iteration.

* "Coordinate descent" version of Jacobi.

* Convergence is guaranteed if $\mathbf{A}$ is striclty row diagonally dominant.

## Successive over-relaxation (SOR)

$$
\small
x_i^{(t+1)} = \frac{\omega}{a_{ii}} \left( b_i - \sum_{j=1}^{i-1} a_{ij} x_j^{(t+1)} - \sum_{j=i+1}^n a_{ij} x_j^{(t)} \right)+ (1-\omega) x_i^{(t)}
$$

* $\omega=1$: Gauss-Seidel; $\omega \in (0, 1)$: underrelaxation; $\omega > 1$: overrelaxation
    - Relaxation in hope of faster convergence
    
* Split $\mathbf{A} = \mathbf{L} + \mathbf{D} + \mathbf{U}$ = (strictly lower triangular) + (diagonal) + (strictly upper triangular) as before.

* Take $\mathbf{M}=\frac{1}{\omega}\mathbf{D}+\mathbf{L}$ (easy to invert, why?) and $\mathbf{K}=\frac{1-\omega}{\omega}\mathbf{D}-\mathbf{U}$:
$$
\begin{split}
(\mathbf{D} + \omega \mathbf{L})\mathbf{x}^{(t+1)} &= [(1-\omega) \mathbf{D} - \omega \mathbf{U}] \mathbf{x}^{(t)} +  \omega \mathbf{b} 
\\
\mathbf{D}\mathbf{x}^{(t+1)} &= (1-\omega) \mathbf{D}\mathbf{x}^{(t)} + \omega ( -\mathbf{U}\mathbf{x}^{(t)} - \mathbf{L}\mathbf{x}^{(t+1)}  + \mathbf{b} )
\\
\mathbf{x}^{(t+1)} &= (1-\omega)\mathbf{x}^{(t)} + \omega \mathbf{D}^{-1} ( -\mathbf{U}\mathbf{x}^{(t)} - \mathbf{L}\mathbf{x}^{(t+1)}  + \mathbf{b} )
\end{split}
$$

## Conjugate gradient method

* CG is the method of choice for solving large, **structured** linear systems $\mathbf{A} \mathbf{x} = \mathbf{b}$, where $\mathbf{A}$ is **positive definite**.

<!--
* Hestenes and Stiefel (1952), [Methods of conjugate gradients for solving linear systems](http://nvlpubs.nist.gov/nistpubs/jres/049/jresv49n6p409_A1b.pdf), _Jounral of Research of the National Bureau of Standards_.
-->

* Solving linear equation $\mathbf{A} \mathbf{x} = \mathbf{b}$, where $\mathbf{A} \in \mathbb{R}^{n \times n}$ is **positive definite**, is equivalent to 
$$
\begin{eqnarray*}
	\text{minimize} \,\, f(\mathbf{x}) = \frac 12 \mathbf{x}^T \mathbf{A} \mathbf{x} - \mathbf{b}^T \mathbf{x}.
\end{eqnarray*}
$$
Denote $\nabla f(\mathbf{x}) = \mathbf{A} \mathbf{x} - \mathbf{b}$.

---

### Motivation: coordinate descent

* Consider minimizing $g(\mathbf{z}) = \frac{1}{2}\mathbf{z}^T\mathbf{z}$, whose minimizer is $\mathbf{0}$.

* **Coordinate descent**: starting from initial $\mathbf{z}_0$, minimize $g(\mathbf{z})$ along coordinate direction $\mathbf{e}_i$ at iteration $i$.
    - $\mathbf{z}_{i+1} = \mathbf{z}_i + t_{i+1}\mathbf{e}_{i+1}$, where
    - $t_{i+1} = \arg\min_t g(\mathbf{z}_i + t \mathbf{e}_{i+1})$
        
    ![](./conjugate_direction.png){width=400}

---

* This procedure achieves the minimum in $n$ iterations ($i=0, 1, \dotsc, n-1$). Also,
$$
    \mathbf{e}_{i+1}^T\mathbf{e}_j = 0, \quad \nabla g(\mathbf{z}_i)^T\mathbf{e}_j = 0, \quad \forall j \le i.
$$

* Due to the spherical symmetry, **we can substitute any set of orthogonal vectors** $\mathbf{u}_1, \dotsc, \mathbf{u}_n$ for $\mathbf{e}_1, \dotsc, \mathbf{e}_n$.

---

### From CD to conjugate directions

* Come back to

$$
\small
\begin{split}
    f(\mathbf{x}) &= \frac{1}{2}\mathbf{x}^T\mathbf{A}\mathbf{x} - \mathbf{b}^T\mathbf{x} \\
        &= \frac{1}{2}(\mathbf{x}-\mathbf{A}^{-1}\mathbf{b})^T\mathbf{A}(\mathbf{x} - \mathbf{A}^{-1}\mathbf{b}) - \frac{1}{2}\mathbf{b}^T\mathbf{A}^{-1}\mathbf{b}
\end{split}        
$$

---

* Coordinate descent in the $\mathbf{x}$-space usually takes too many iterations:

![](./coordinate_descent.png){width=400}

---

* Change of variables $\mathbf{z} = \mathbf{A}^{-1/2}\mathbf{x} - \mathbf{A}^{-1}\mathbf{b}$ yiels
$$
\small
\begin{split}
    f(\mathbf{z}) &= f(\mathbf{A}^{-1/2}\mathbf{x} - \mathbf{A}^{-1}\mathbf{b}) \\
        &= \frac{1}{2}\mathbf{z}^T\mathbf{z} - \frac{1}{2}\mathbf{b}^T\mathbf{A}^{-1}\mathbf{b} = g(\mathbf{z}) - \frac{1}{2}\mathbf{b}^T\mathbf{A}^{-1}\mathbf{b} 
\end{split}
$$
Thus the problem reduces to minimizing $g(\mathbf{z})$ in the transformed space.

---

* Now translate what happens in the $z$-space to the $x$-space:

| Method            | $z$-space                | $x$-space                 |
| :---------------: | :--------------------- | :---------------------- |
| Direction         | $\mathbf{u}_i$         | $\mathbf{v}_i=\mathbf{A}^{-1/2}\mathbf{u}_i$ |
| Orthogonality     | $\mathbf{u}_{i+1}^T\mathbf{u}_j=0$, $\forall j\le i$ | $\mathbf{v}_{i+1}^T\mathbf{A}\mathbf{v}_j=0$, $\forall j\le i$ |
| Optimality        | $\nabla g(\mathbf{z}_i)^T\mathbf{u}_j=0$, $\forall j\le i$ | $(\mathbf{A}^{-1/2}\nabla f(\mathbf{x}_i))^T\mathbf{A}^{1/2}\mathbf{v}_j=\nabla f(\mathbf{x}_i)^T\mathbf{v}_j$,  $\forall j\le i$ |

The last condition is due to $\nabla g(\mathbf{z}) = \mathbf{A}^{-1/2}\nabla f(\mathbf{x})$.
    
* Condition $\mathbf{v}_i^T\mathbf{A}\mathbf{v}_j=0$ is called $\mathbf{A}$-orthogonality or *conjugacy*.
    - Vectors $\mathbf{v}_1, \dotsc, \mathbf{v}_n$ are called **conjugate directions**.
    
* **Theorem**: searching along the conjugate directions, $\mathbf{x}_i$ converges to the solution in **at most** $n$ steps (in exact arithmetic).

---

### CG algorithm (basic version) {.smaller}

0. Given $\mathbf{x}_0$
0. Initialize: $\mathbf{r}_0 \gets \mathbf{A} \mathbf{x}_0 - \mathbf{b}$, $\mathbf{v}_1 \gets - \mathbf{r}_0$, $\alpha_1 \gets 0$, $i \gets 1$
0. While $\mathbf{r}_i \ne \mathbf{0}$
    1. $t_{i} \gets - {\mathbf{r}_{i-1}^T \mathbf{v}_{i}}/{\mathbf{v}_{i}^T \mathbf{A} \mathbf{v}_{i}}$
    2. $\mathbf{x}_{i} \gets \mathbf{x}_{i-1} + t_{i} \mathbf{v}_{i}$
    3. $\mathbf{r}_{i} \gets \mathbf{A} \mathbf{x}_{i} - \mathbf{b}$
    4. $\alpha_{i+1} \gets {\mathbf{r}_i^T \mathbf{A} \mathbf{v}_i}/{\mathbf{v}_i^T \mathbf{A} \mathbf{v}_i}$
    5. $\mathbf{v}_{i+1} \gets - \mathbf{r}_i + \alpha_{i+1} \mathbf{v}_i$        
    6. $i \gets i+1$

---

* Note $\mathbf{r}_i = \nabla f(\mathbf{x_i})$.
* Coordinate descent: $\mathbf{x}_i = \mathbf{x}_{i-1} + t_i \mathbf{v}_i$ for some $t_i$. Hence
$$
    \mathbf{r}_i = \mathbf{A}\mathbf{x}_i - \mathbf{b} = \mathbf{A}\mathbf{x}_{i-1} - \mathbf{b} + t_i\mathbf{A}\mathbf{v}_i = \mathbf{r}_{i-1} + t_i\mathbf{A}\mathbf{v}_i
$$
* From the optimaltity conditions $\mathbf{r}_i^T\mathbf{v}_j=0$, $\forall j\le i$. Therefore
$$
    \mathbf{v}_i^T\mathbf{r}_i = 0 = \mathbf{v}_i^T\mathbf{r}_{i-1} + t_i\mathbf{v}_i^T\mathbf{A}\mathbf{v}_i,
$$
yielding $t_i = - \mathbf{v}_i^T\mathbf{r}_{i-1}/\mathbf{v}_i^T\mathbf{A}\mathbf{v}_i$.

---

* Conjugate directions are constructed recursively, observing the conjugacy conditions $\mathbf{v}_{i+1}^T\mathbf{A}\mathbf{v}_j$ for all $j\le i$:
$$
    \mathbf{v}_{i+1} = -\nabla f(\mathbf{y}_{i}) + \alpha_{i+1}\mathbf{v}_{i} = -\mathbf{r}_{i} + \alpha_{i+1}\mathbf{v}_i.
$$
* Choosing initial direction $\mathbf{v}_1 = -\mathbf{r}_0$ yields $\text{span}(\mathbf{r}_0,\dotsc,\mathbf{r}_{i-1}) = \text{span}(\mathbf{v}_1,\dotsc,\mathbf{v}_i)$.
* Using this, it can be shown that $\alpha_{i+1} = \frac{\mathbf{r}_i^T \mathbf{A} \mathbf{v}_i}{\mathbf{v}_i^T \mathbf{A} \mathbf{v}_i}$ makes the sequence $\{\mathbf{v}_1, \dotsc, \mathbf{v}_{i+1} \}$ $\mathbf{A}$-orthogonal (HW).

---

### CG algorithm (economical version): {.smaller}

0. Given $\mathbf{x}_0$
0. Initialize: $\mathbf{r}_0 \gets \mathbf{A} \mathbf{x}_0 - \mathbf{b}$, $\mathbf{v}_1 \gets - \mathbf{r}_0$, $\alpha_1 \gets 0$, $i \gets 1$
0. While $\mathbf{r}_i \ne \mathbf{0}$
    1. $t_{i} \gets  {\mathbf{r}_{i-1}^T \mathbf{r}_{i}}/{\mathbf{v}_{i}^T \mathbf{A} \mathbf{v}_{i}}$
    2. $\mathbf{x}_{i} \gets \mathbf{x}_{i-1} + t_{i}\mathbf{v}_{i}$
    3. $\mathbf{r}_{i} \gets \mathbf{r}_{i-1} + t_{i}\mathbf{A}\mathbf{v}_{i}$
    4. $\boxed{\alpha_{i+1} \gets {\mathbf{r}_i^T \mathbf{r}_i}/{\mathbf{r}_{i-1}^T \mathbf{r}_{i-1}}}$
    5. $\mathbf{v}_{i+1} \gets - \mathbf{r}_i + \alpha_{i+1} \mathbf{v}_i$        
    6. $i \gets i+1$
    
    Because $\mathbf{r}_i^T\mathbf{r}_j = 0$, $\forall j < i$ (why?) yields $\mathbf{r}_i^T\mathbf{r}_i = t_i \mathbf{r}_i^T\mathbf{A}\mathbf{r}_i$ and $\mathbf{r}_{i-1}^T\mathbf{r}_{i-1} = -\mathbf{v}_i^T\mathbf{r}_{i-1}$.

* Saves one matrix-vector multiplication: $\mathbf{A} \mathbf{v}_{i}$.

---

### Krylov subspaces

* $\{\mathbf{r}_0,\dotsc,\mathbf{r}_{k-1}\}$ is contained in the **Krylov subspace** of degree $k$ for $\mathbf{r}_0$:

$$
        {\cal K}^k(\mathbf{A}, \mathbf{r}_0) = \text{span} \{\mathbf{r}_0, \mathbf{A}\mathbf{r}_0, \mathbf{A}^2 \mathbf{r}_0, \dotsc, \mathbf{A}^{k-1} \mathbf{r}_0 \}.
$$

* $\{\mathbf{v}_1,\ldots,\mathbf{v}_{k}\}$ is contained in ${\cal K}^k(\mathbf{A}, \mathbf{r}_0)$. 

    
* **Theorem**: If $\mathbf{A}$ has $r$ distinct eigenvalues, $\mathbf{x}_k$ converges to solution $\mathbf{x}^\star=\mathbf{A}^{-1}\mathbf{b}$ in at most $r$ steps.

---

* Two important bounds for CG:

    Let $0 < \lambda_1 \le \cdots \le \lambda_n$ be the ordered eigenvalues of $\mathbf{A}$.  
    
$$
\small
\begin{eqnarray*}
    \|\mathbf{x}^{(t+1)} - \mathbf{x}^*\|_{\mathbf{A}}^2 &\le& \left( \frac{\lambda_{n-t} - \lambda_1}{\lambda_{n-t} + \lambda_1} \right)^2 \|\mathbf{x}^{(0)} - \mathbf{x}^*\|_{\mathbf{A}}^2 \\
    \|\mathbf{x}^{(t+1)} - \mathbf{x}^*\|_{\mathbf{A}}^2 &\le& 2 \left( \frac{\sqrt{\kappa(\mathbf{A})}-1}{\sqrt{\kappa(\mathbf{A})}+1} \right)^{t} \|\mathbf{x}^{(0)} - \mathbf{x}^*\|_{\mathbf{A}}^2,
\end{eqnarray*}
$$

  where $\kappa(\mathbf{A}) = \lambda_n/\lambda_1$ is the condition number of $\mathbf{A}$ (why?).

* Messages:
    * Roughly speaking, if the eigenvalues of $\mathbf{A}$ occur in $r$ distinct clusters, the CG iterates will _approximately_ solve the problem after $O(r)$ steps.  
    * $\mathbf{A}$ with a small condition number ($\lambda_1 \approx \lambda_n$) converges fast.
    

## Preconditioned CG

* **Preconditioning**: Change of variables $\widehat{\mathbf{x}} = \mathbf{C} \mathbf{x}$ via a nonsingular $\mathbf{C}$ and solve
$$
	(\mathbf{C}^{-T} \mathbf{A} \mathbf{C}^{-1}) \widehat{\mathbf{x}} = \mathbf{C}^{-T} \mathbf{b}.
$$
Choose $\mathbf{C}$ such that 
    * $\mathbf{C}^{-T} \mathbf{A} \mathbf{C}^{-1}$ has small condition number, or 
    * $\mathbf{C}^{-T} \mathbf{A} \mathbf{C}^{-1}$ has clustered eigenvalues
    * Inexpensive solution of $\mathbf{C}^T \mathbf{C} \mathbf{y} = \mathbf{r}$
    
* PCG does not make use of $\mathbf{C}$ explicitly, but rather the matrix $\mathbf{M} = \mathbf{C}^T \mathbf{C}$.

---

### PCG algorithm

0. Given $\mathbf{x}_0$, preconditioner $\mathbf{M}$
0. Initialize: $\mathbf{r}_0 \gets \mathbf{A} \mathbf{x}_0 - \mathbf{b}$
0. Solve $\mathbf{M} \mathbf{y}_0 = \mathbf{r}_0$ for $\mathbf{y}_0$
0. Initialize: $\mathbf{v}_1 \gets - \mathbf{r}_0$, $\alpha_1 \gets 0$, $i \gets 1$
0. While $\mathbf{r}_i \ne \mathbf{0}$
    1. $t_{i} \gets - {\mathbf{y}_{i-1}^T \mathbf{r}_{i}}/{\mathbf{v}_{i}^T \mathbf{A} \mathbf{v}_{i}}$
    2. $\mathbf{x}_{i} \gets \mathbf{x}_{i-1} + t_{i}\mathbf{v}_{i}$
    3. $\mathbf{r}_{i} \gets \mathbf{r}_{i-1} + t_{i}\mathbf{A}\mathbf{v}_{i}$
    3. Solve $\mathbf{M} \mathbf{y}_{i}=\mathbf{r}_{i}$ for $\mathbf{y}_{i}$
    4. $\alpha_{i+1} \gets - {\mathbf{r}_i^T \mathbf{y}_i}/{\mathbf{r}_{i-1}^T \mathbf{r}_{i-1}}$
    5. $\mathbf{v}_{i+1} \gets - \mathbf{y}_i + \alpha_{i+1} \mathbf{v}_i$        
    6. $i \gets i+1$
    
    Remark: Only extra cost of solving the linear system $\mathbf{M} \mathbf{y} = \mathbf{r}$.
    
---

* Preconditioning is more like an art than science. Some choices include     
    * Incomplete Cholesky. $\mathbf{A} \approx \tilde{\mathbf{L}} \tilde{\mathbf{L}}^T$, where $\tilde{\mathbf{L}}$ is a sparse approximate Cholesky factor. Then $\tilde{\mathbf{L}}^{-1} \mathbf{A} \tilde{\mathbf{L}}^{-T} \approx \mathbf{I}$ (perfectly conditioned) and $\mathbf{M} \mathbf{y} = \tilde{\mathbf{L}} \tilde {\mathbf{L}}^T \mathbf{y} = \mathbf{r}$ is easy to solve.  
    * Banded pre-conditioners.  
    * Choose $\mathbf{M}$ as a coarsened version of $\mathbf{A}$.
    * Subject knowledge. Knowing the structure and origin of a problem is often the key to devising efficient pre-conditioner. For example, see [Lee, Won, Lim, and Yoon (2017)](https://doi.org/10.1080/10618600.2017.1328363) for large-scale image-based regression.

---

### Application to a fusion problem in physics

[The incomplete Cholesky—conjugate gradient method for the iterative solution of systems of linear equations](http://www.sciencedirect.com/science/article/pii/0021999178900980?via%3Dihub) by D. Kershaw:

| Method                                 | Number of Iterations |
|----------------------------------------|----------------------|
| Gauss-Seidel                           | 208,000              |
| Block SOR methods                      | 765                  |
| Incomplete Cholesky CG                 | 25                   |



> **The structure should be exploited whenever possible in solving a problem.** 

## Numerical examples

`MatrixDepot.jl` is an extensive collection of test matrices in Julia. After installation, we can check available test matrices by `mdinfo()`.

### Poisson linear system

In [3]:
using BenchmarkTools, IterativeSolvers, LinearAlgebra, MatrixDepot, Random

Random.seed!(280)

n = 100
# Poisson matrix of dimension n^2=10000, pd and sparse
A = matrixdepot("poisson", n)
@show typeof(A)
# dense matrix representation of A
Afull = convert(Matrix, A)
@show typeof(Afull)
# sparsity level
count(!iszero, A) / length(A)

[ Info: verify download of index files...
[ Info: reading database
[ Info: adding metadata...
[ Info: adding svd data...
[ Info: writing database
[ Info: used remote sites are sparse.tamu.edu with MAT index and math.nist.gov with HTML index


typeof(A) = SparseArrays.SparseMatrixCSC{Float64, Int64}
typeof(Afull) = Matrix{Float64}


0.000496

---

In [4]:
using UnicodePlots
spy(A)

          ┌──────────────────────────────────────────┐    
        1 │⠻⣦⡀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀│ > 0
          │⠀⠈⠻⣦⡀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀│ < 0
          │⠀⠀⠀⠈⠻⣦⡀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀│    
          │⠀⠀⠀⠀⠀⠈⠻⣦⡀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀│    
          │⠀⠀⠀⠀⠀⠀⠀⠈⠻⣦⡀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀│    
          │⠀⠀⠀⠀⠀⠀⠀⠀⠀⠈⠻⣦⡀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀│    
          │⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠈⠻⣦⡀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀│    
          │⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠈⠻⣦⡀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀│    
          │⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠈⠻⣦⡀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀│    
          │⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠈⠻⣦⡀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀│    
          │⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠈⠻⣦⡀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀│    
          │⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠈⠻⣦⡀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀│    
          │⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠈⠻⣦⡀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀│    
          │⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠈⠻⣦⡀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀│    
          │⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠈⠻⣦⡀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀│    
          │⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠈⠻⣦⡀⠀⠀⠀⠀⠀⠀⠀⠀⠀│  

In [5]:
# storage difference
Base.summarysize(A), Base.summarysize(Afull)

(873768, 800000040)

---

### Matrix-vector muliplication

In [6]:
# randomly generated vector of length n^2
b = randn(n^2)
# dense matrix-vector multiplication
@benchmark $Afull * $b

BenchmarkTools.Trial: 119 samples with 1 evaluation.
 Range (min … max):  39.458 ms … 57.874 ms  ┊ GC (min … max): 0.00% … 0.00%
 Time  (median):     41.578 ms              ┊ GC (median):    0.00%
 Time  (mean ± σ):   41.978 ms ±  1.886 ms  ┊ GC (mean ± σ):  0.00% ± 0.00%

         ▆▆█▃▄ ▁                                               
  ▃▁▁▁▁▄▃█████▇█▇▇▄▄▁▄▃▁▁▄▁▃▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▃ ▃
  39.5 ms         Histogram: frequency by time        51.5 ms <

 Memory estimate: 78.17 KiB, allocs estimate: 2.

In [7]:
# sparse matrix-vector multiplication
@benchmark $A * $b

BenchmarkTools.Trial: 10000 samples with 1 evaluation.
 Range (min … max):   44.813 μs …   5.904 ms  ┊ GC (min … max): 0.00% … 97.79%
 Time  (median):      87.975 μs               ┊ GC (median):    0.00%
 Time  (mean ± σ):   100.695 μs ± 224.062 μs  ┊ GC (mean ± σ):  9.07% ±  4.02%

  ▃▂▂           ▄▇███▇▇▆▅▄▃▃▂▁  ▁ ▁ ▁                           ▃
  ████▇▆▅▆▅▆▅▄▁▇███████████████▇███▇████▇▇▇█▇▅▆▇█▇█▇▇▆▆▇▇▆▇▆▅▆▇ █
  44.8 μs       Histogram: log(frequency) by time        189 μs <

 Memory estimate: 78.17 KiB, allocs estimate: 2.

---

### Dense solve via Cholesky

In [8]:
# record the Cholesky solution
xchol = cholesky(Afull) \ b;

In [9]:
# dense solve via Cholesky

@benchmark cholesky($Afull) \ $b

BenchmarkTools.Trial: 2 samples with 1 evaluation.
 Range (min … max):  3.272 s …    3.448 s  ┊ GC (min … max): 1.53% … 0.01%
 Time  (median):     3.360 s               ┊ GC (median):    0.75%
 Time  (mean ± σ):   3.360 s ± 123.938 ms  ┊ GC (mean ± σ):  0.75% ± 1.08%

  █                                                        █  
  █▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁█ ▁
  3.27 s         Histogram: frequency by time         3.45 s <

 Memory estimate: 763.02 MiB, allocs estimate: 4.

---

### Iterative solvers 
The [`IterativeSolvers.jl`](https://github.com/JuliaMath/IterativeSolvers.jl) package implements most commonly used iterative solvers.

#### Jacobi solver

In [10]:
xjacobi = jacobi(A, b)
@show norm(xjacobi - xchol)

norm(xjacobi - xchol) = 498.7144076230417


498.7144076230417

* It seems that Jacobi solver doesn't give the correct answer.

---

* [Documentation](https://juliamath.github.io/IterativeSolvers.jl/dev/linear_systems/stationary/#Jacobi-1) reveals that the default value of `maxiter` is 10. A couple trial runs shows that 30000 Jacobi iterations are enough to get an accurate solution.

In [11]:
xjacobi = jacobi(A, b, maxiter=30000)
@show norm(xjacobi - xchol)

norm(xjacobi - xchol) = 1.626836219074868e-5


1.626836219074868e-5

In [12]:
@benchmark jacobi($A, $b, maxiter=30000)

BenchmarkTools.Trial: 3 samples with 1 evaluation.
 Range (min … max):  1.822 s …   1.994 s  ┊ GC (min … max): 0.00% … 0.00%
 Time  (median):     1.843 s              ┊ GC (median):    0.00%
 Time  (mean ± σ):   1.886 s ± 93.775 ms  ┊ GC (mean ± σ):  0.00% ± 0.00%

  █      █                                                █  
  █▁▁▁▁▁▁█▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁█ ▁
  1.82 s         Histogram: frequency by time        1.99 s <

 Memory estimate: 2.06 MiB, allocs estimate: 30008.

---

#### Gauss-Seidel

In [13]:
# Gauss-Seidel solution is fairly close to Cholesky solution after 15000 iters
xgs = gauss_seidel(A, b, maxiter=15000)
@show norm(xgs - xchol)

norm(xgs - xchol) = 1.429290496951622e-5


1.429290496951622e-5

In [14]:
@benchmark gauss_seidel($A, $b, maxiter=15000)

BenchmarkTools.Trial: 4 samples with 1 evaluation.
 Range (min … max):  1.550 s …   1.592 s  ┊ GC (min … max): 0.00% … 0.00%
 Time  (median):     1.578 s              ┊ GC (median):    0.00%
 Time  (mean ± σ):   1.574 s ± 17.958 ms  ┊ GC (mean ± σ):  0.00% ± 0.00%

  █                                  █   █                █  
  █▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁█▁▁▁█▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁█ ▁
  1.55 s         Histogram: frequency by time        1.59 s <

 Memory estimate: 156.58 KiB, allocs estimate: 7.

---

#### SOR

In [15]:
# symmetric SOR with ω=0.75
xsor = ssor(A, b, 0.75, maxiter=10000)
@show norm(xsor - xchol)

norm(xsor - xchol) = 0.0002980091296489782


0.0002980091296489782

In [16]:
@benchmark sor($A, $b, 0.75, maxiter=10000)

BenchmarkTools.Trial: 5 samples with 1 evaluation.
 Range (min … max):  1.220 s …   1.263 s  ┊ GC (min … max): 0.00% … 0.00%
 Time  (median):     1.226 s              ┊ GC (median):    0.00%
 Time  (mean ± σ):   1.235 s ± 18.043 ms  ┊ GC (mean ± σ):  0.00% ± 0.00%

  █    █ █                       █                        █  
  █▁▁▁▁█▁█▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁█▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁█ ▁
  1.22 s         Histogram: frequency by time        1.26 s <

 Memory estimate: 547.27 KiB, allocs estimate: 20009.

---

#### Conjugate Gradient

In [17]:
# conjugate gradient
xcg = cg(A, b)
@show norm(xcg - xchol)

norm(xcg - xchol) = 1.2232567404794239e-5


1.2232567404794239e-5

In [18]:
@benchmark cg($A, $b)

BenchmarkTools.Trial: 252 samples with 1 evaluation.
 Range (min … max):  17.134 ms … 43.064 ms  ┊ GC (min … max): 0.00% … 0.00%
 Time  (median):     19.444 ms              ┊ GC (median):    0.00%
 Time  (mean ± σ):   19.862 ms ±  2.413 ms  ┊ GC (mean ± σ):  0.12% ± 1.43%

    ▁     ▂▃▃▃▄▅▁█    ▂                                        
  ▄▃█▆▆▆▅▆████████▆█▆██▅▇▃▁▄▃▃▃▄▁▁▁▄▁▃▁▁▁▃▃▃▁▁▃▁▃▁▁▁▃▁▁▁▁▁▁▁▃ ▃
  17.1 ms         Histogram: frequency by time        27.2 ms <

 Memory estimate: 313.39 KiB, allocs estimate: 16.

---

### Example of PCG

[Preconditioners.jl](https://github.com/mohamed82008/Preconditioners.jl) wraps a bunch of preconditioners.

We use the Wathen matrix (sparse and positive definite) as a test matrix.

In [19]:
using BenchmarkTools, MatrixDepot, IterativeSolvers, LinearAlgebra, SparseArrays

# Wathen matrix of dimension 30401 x 30401
A = matrixdepot("wathen", 100);
spy(A)

          ┌──────────────────────────────────────────┐    
        1 │⠻⣦⡀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀│ > 0
          │⠀⠈⠻⣦⡀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀│ < 0
          │⠀⠀⠀⠈⠻⣦⡀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀│    
          │⠀⠀⠀⠀⠀⠈⠻⣦⡀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀│    
          │⠀⠀⠀⠀⠀⠀⠀⠈⠻⣦⡀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀│    
          │⠀⠀⠀⠀⠀⠀⠀⠀⠀⠈⠻⣦⡀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀│    
          │⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠈⠻⣦⡀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀│    
          │⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠈⠻⣦⡀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀│    
          │⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠈⠻⣦⡀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀│    
          │⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠈⠻⣦⡀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀│    
          │⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠈⠻⣦⡀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀│    
          │⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠈⠻⣦⡀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀│    
          │⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠈⠻⣦⡀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀│    
          │⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠈⠻⣦⡀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀│    
          │⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠈⠻⣦⡀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀│    
          │⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠈⠻⣦⡀⠀⠀⠀⠀⠀⠀⠀⠀⠀│  

---

In [20]:
# sparsity level
count(!iszero, A) / length(A)


0.0005102687577359558

In [21]:
# rhs
b = ones(size(A, 1))
# solve Ax=b by CG
xcg = cg(A, b);
@benchmark cg($A, $b)


BenchmarkTools.Trial: 30 samples with 1 evaluation.
 Range (min … max):  142.752 ms … 282.725 ms  ┊ GC (min … max): 0.00% … 0.00%
 Time  (median):     164.991 ms               ┊ GC (median):    0.00%
 Time  (mean ± σ):   168.073 ms ±  25.266 ms  ┊ GC (mean ± σ):  0.00% ± 0.00%

  █    ▂         ▂                                               
  █▁▁▁██▅▅██▅▁████▅▅▅▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▅ ▁
  143 ms           Histogram: frequency by time          283 ms <

 Memory estimate: 951.14 KiB, allocs estimate: 16.

---

Compute the incomplete cholesky preconditioner:

In [22]:
using Preconditioners
@time p = CholeskyPreconditioner(A, 2)


  0.977141 seconds (1.02 M allocations: 88.303 MiB, 2.17% gc time, 94.85% compilation time)


CholeskyPreconditioner{LimitedLDLFactorizations.LimitedLDLFactorization{Float64, Int64, Vector{Int64}, Vector{Int64}}}(LimitedLDLFactorizations.LimitedLDLFactorization{Float64, Int64, Vector{Int64}, Vector{Int64}}(true, 30401, [1, 13, 25, 37, 54, 65, 77, 89, 101, 118  …  265018, 265021, 265025, 265028, 265032, 265035, 265037, 265038, 265038, 265038], [3, 4, 5, 11, 12, 1557, 1558, 1559, 1599, 4639  …  30394, 30395, 30396, 30396, 30397, 30398, 30398, 30399, 30400, 30400], [3, 4, 5, 11, 12, 1557, 1558, 1559, 1599, 4639  …  30397, 30398, 30399, 30400, 30398, 30399, 30400, 30399, 30400, 30400], [0.467273868045779, -0.18750000000000003, 0.15772613195422108, 0.12618090556337683, -0.06309045278168841, 0.4672738680457789, -0.1875, -0.06309045278168843, 0.15772613195422105, -0.18690954721831163  …  -0.20746436142724575, -0.19174225470713907, 0.09492563158511769, -0.2080503616036747, -0.18733644439993166, 0.09756055779983207, -0.22499574760650562, -0.1694477428123921, 0.10047680886206374, -0.2537

PCG:

In [23]:
# solver Ax=b by PCG
xpcg = cg(A, b, Pl=p)
# same answer?
norm(xcg - xpcg)


4.171901128059577e-7

In [24]:
# PCG is >10 fold faster than CG
@benchmark cg($A, $b, Pl=$p)


BenchmarkTools.Trial: 200 samples with 1 evaluation.
 Range (min … max):  23.598 ms … 32.825 ms  ┊ GC (min … max): 0.00% … 24.52%
 Time  (median):     24.739 ms              ┊ GC (median):    0.00%
 Time  (mean ± σ):   25.054 ms ±  1.361 ms  ┊ GC (mean ± σ):  0.32% ±  2.47%

      ▄▆ ▇▅█▁▂                                                 
  ▄▄▄█████████▄▆▄▃▄▅▃▃▁▁▃▃▁▁▁▁▃▁▃▃▃▁▁▁▁▁▁▁▃▁▁▁▁▃▃▁▁▁▁▁▁▁▁▁▁▁▃ ▃
  23.6 ms         Histogram: frequency by time        31.8 ms <

 Memory estimate: 951.14 KiB, allocs estimate: 16.

## Other Krylov subspace methods

* We leant about CG/PCG, which is for solving $\mathbf{A} \mathbf{x} = \mathbf{b}$, $\mathbf{A}$ pd.

* **MINRES (minimum residual method)**: symmetric *indefinite* $\mathbf{A}$.

* **GMRES (generalized minimum residual method)**: current _de facto_ method for unsymmetric $\mathbf{A}$, e.g., PageRank problem.

* **LSQR**: least square problem $\min \|\mathbf{y} - \mathbf{X} \beta\|_2^2$. Algebraically equivalent to applying CG to the normal equation $(\mathbf{X}^T \mathbf{X} + \lambda^2 I) \beta = \mathbf{X}^T \mathbf{y}$.

* **LSMR**: least square problem $\min \|\mathbf{y} - \mathbf{X} \beta\|_2^2$. Algebraically equivalent to applying MINRES to the normal equation $(\mathbf{X}^T \mathbf{X} + \lambda^2 I) \beta = \mathbf{X}^T \mathbf{y}$.

---

#### Least squares example

In [25]:
using BenchmarkTools, IterativeSolvers, LinearAlgebra, Random, SparseArrays

Random.seed!(280) # seed
n, p = 10000, 5000
X = sprandn(n, p, 0.001) # iid standard normals with sparsity 0.01
β = ones(p)
y = X * β + randn(n)

10000-element Vector{Float64}:
  1.7812991965741505
 -2.0785423294216105
 -1.0481734841421668
 -0.6511565465968179
 -1.3447885390086307
  1.7177040773697279
 -6.074715684464104
  2.5216398057120526
  1.7700788885316854
  1.0651756075872156
  1.6681474872248563
  0.200997252202254
 -0.08163839046853827
  ⋮
  1.0982950170409602
  3.4768880180740362
  2.6757792009942705
 -0.14763168770175514
  1.2755620601224047
  0.0764091156627229
  5.7313325719704515
  1.8639652711071197
  0.42148290416775736
  5.4001228554307135
  1.3082807058105932
  3.151127504155086

In [26]:
# least squares by QR
betahat_qr = X \ y
@benchmark $X \ $y

BenchmarkTools.Trial: 1 sample with 1 evaluation.
 Single result which took 6.723 s (0.06% GC) to evaluate,
 with a memory estimate of 1.55 GiB, over 185 allocations.

---

In [27]:
# LSQR
betahat_lsqr = lsqr(X, y)
@show norm(betahat_lsqr - betahat_qr)
@benchmark lsqr($X, $y)

norm(betahat_lsqr - betahat_qr) = 0.00010289622980521999


BenchmarkTools.Trial: 114 samples with 1 evaluation.
 Range (min … max):  37.596 ms … 65.122 ms  ┊ GC (min … max): 0.00% … 21.82%
 Time  (median):     41.560 ms              ┊ GC (median):    0.00%
 Time  (mean ± σ):   44.008 ms ±  6.543 ms  ┊ GC (mean ± σ):  6.12% ± 10.12%

  ▁ ▄▄▂ ▅▅▇█ ▄                                                 
  █████▆████▃█▆█▆▅▃▃▅▁▁▁▁▁▃▁▃▁▁▁▃▁▃▁▃▁▆▃█▁▃▁▃▁▃▁▁▁▁▃▃▁▃▃▁▁▃▁▃ ▃
  37.6 ms         Histogram: frequency by time        63.4 ms <

 Memory estimate: 16.63 MiB, allocs estimate: 1724.

In [29]:
# LSMR
betahat_lsmr = lsmr(X, y)
@show norm(betahat_lsmr - betahat_qr)
@benchmark lsmr($X, $y)

norm(betahat_lsmr - betahat_qr) = 1.1712840250276548


BenchmarkTools.Trial: 206 samples with 1 evaluation.
 Range (min … max):  21.596 ms … 48.148 ms  ┊ GC (min … max): 0.00% … 36.74%
 Time  (median):     22.755 ms              ┊ GC (median):    0.00%
 Time  (mean ± σ):   24.270 ms ±  3.952 ms  ┊ GC (mean ± σ):  3.63% ±  9.05%

  ▁▃█▇▆▅▂▂   ▁   ▁                                             
  ████████▇▅██████▇▅▁▆▆▅▁▅▁▁▅▁▁▅▁▁▅▁▁▁▅▁▅▁▁▁▁▁▁▁▁▁▁▁▁▆▇▅▁▁▆▅▆ ▆
  21.6 ms      Histogram: log(frequency) by time      37.3 ms <

 Memory estimate: 5.54 MiB, allocs estimate: 706.

## Further reading

* Chapter 5 of [Numerical Optimization](https://link.springer.com/book/10.1007/978-0-387-40065-5) by Jorge Nocedal and Stephen Wright (1999).

* Sections 11.3-11.5 of [Matrix Computations](https://ucla.worldcat.org/title/matrix-computations/oclc/824733531&referer=brief_results) by Gene Golub and Charles Van Loan (2013).

* Hestenes and Stiefel (1952), [Methods of conjugate gradients for solving linear systems](http://nvlpubs.nist.gov/nistpubs/jres/049/jresv49n6p409_A1b.pdf), _Jounral of Research of the National Bureau of Standards_.